### 爬取周星驰相关的100位演员关系网络

In [2]:
import requests
import json
#周星驰演员id=1048026
actor_id = '1048026'
access_key = '0df993c66c0c636e29ecbb5344252a4a' 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
res = requests.get(f'http://api.douban.com/v2/movie/celebrity/{actor_id}?apikey={access_key}',headers=headers)
json.loads(res.content.decode('utf-8'))

{'website': '',
 'mobile_url': 'https://movie.douban.com/celebrity/1048026/mobile',
 'aka_en': ['Sing-Chi Chow (本名)', 'Sing Jai (昵称)'],
 'name': '周星驰',
 'works': [{'roles': ['演员'],
   'subject': {'rating': {'max': 10,
     'average': 9.2,
     'details': {'1': 1948.0,
      '3': 45854.0,
      '2': 3542.0,
      '5': 517327.0,
      '4': 202411.0},
     'stars': '45',
     'min': 0},
    'genres': ['喜剧', '爱情', '奇幻'],
    'title': '大话西游之大圣娶亲',
    'casts': [{'avatars': {'small': 'http://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p47421.jpg',
       'large': 'http://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p47421.jpg',
       'medium': 'http://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p47421.jpg'},
      'name_en': 'Stephen Chow',
      'name': '周星驰',
      'alt': 'https://movie.douban.com/celebrity/1048026/',
      'id': '1048026'},
     {'avatars': {'small': 'http://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p45481.jpg',
  

In [3]:
#批量获取接口数据的方法
def get(url,headers):
    while True:
        try:
            res = requests.get(url,headers = headers, timeout = 5)
            return json.loads(res.content.decode('utf-8'))
        except requests.exceptions.Timeout:
            time.sleep(1)

In [6]:
import queue
import os
import requests
import json
access_key = '0df993c66c0c636e29ecbb5344252a4a'  # api key
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
# 演员队列初始为周星驰 id
actorQ = queue.Queue()
actorQ.put('1048026')#申请获得互斥锁，获得后，如果队列未满，则向队列中添加数据，并通知notify其它阻塞的某个线程，唤醒等待获取require互斥锁。如果队列已满，则会wait等待。最后处理完成后释放互斥锁。

total = 100
# 创建文件夹方便整理数据
if not os.path.exists('actors'):
    os.mkdir('actors')

while not actorQ.empty():
    actor_id = actorQ.get() #从队列中获取任务，并且从队列中移除此任务。
    actor_path = os.path.join('actors', str(actor_id)+'.json')
    # 如果已爬取则从本地读取
    if os.path.exists(actor_path):
        # 读取文件
        with open(actor_path,encoding = 'utf-8') as f:
            json_content = json.load(f)
    # 否则爬取数据
    else:
        json_content = get(f'http://api.douban.com/v2/movie/celebrity/{actor_id}?apikey={access_key}',headers)
        # 爬完后写入本地
        with open(actor_path, 'w',encoding = 'utf-8') as f:
            f.write(json.dumps(json_content, indent=4, ensure_ascii=False))

    # 遍历该演员的电影的其他演员，并将 id 加入 actorQ 重新遍历
    for movie in json_content['works']:
        for x in movie['subject']['casts']:
            # 如果没有该演员的 id 则跳过
            if x['id'] == None:
                continue
            actorQ.put(x['id'])
    if len(os.listdir('actors')) > total:
        print('爬取完毕')
        break

爬取完毕


In [ ]:
#将上面得到的 100 位演员的信息合并为一个 JSON 文件
actors = {}
for json_file in tqdm_notebook(os.listdir('actors')):
    json_file_path = os.path.join('actors', json_file)
    with open(json_file_path,encoding = 'utf-8') as f:
        json_content = json.load(f)
        actors[json_content['id']] = json_content
# 写入文件
with open('actors.json', 'w',encoding = 'utf-8') as f:
    f.write(json.dumps({'actors': actors}, indent=4, ensure_ascii=False))    

### 绘图分析周星驰相关的 100 位演员关系网络

In [8]:
# 加载演员数据
with open('actors.json',encoding = 'utf-8') as f:
    actors_data = json.load(f)['actors']

In [10]:
import itertools
# 有过合作的演员的组合
has_cooperated_actors = []
# 所有演员的名字
all_names = []
# 统计演员一起出现的电影
for actor_id in actors_data:
    # 演员演过的电影
    movies = actors_data[actor_id]['works']
    for movie in movies:
        # 获取电影的主演的 id
        # 有一些演员的ID为空，大部分都是一些名气不大的演员，所以影响不大
        movie_actors = [actor['name'] for actor in movie['subject']
                        ['casts'] if actor['id'] in actors_data.keys()]
        all_names += movie_actors
        has_cooperated_actors += list(itertools.combinations(movie_actors, 2))
has_cooperated_actors = list(set(has_cooperated_actors))
all_names = list(set(all_names))
has_cooperated_actors

[('温兆伦', '洪欣'),
 ('张柏芝', '陈奕迅'),
 ('周润发', '刘嘉玲'),
 ('巩俐', '陈百祥'),
 ('黄杏秀', '汤镇业'),
 ('张国荣', '梁朝伟'),
 ('周润发', '吕良伟'),
 ('黄日华', '陈秀雯'),
 ('吴孟达', '张敏'),
 ('周润发', '吴孟达'),
 ('周润发', '赵雅芝'),
 ('江华', '汤镇业'),
 ('严屹宽', '富大龙'),
 ('周星驰', '巩俐'),
 ('朱茵', '林倩萍'),
 ('张柏芝', '卢巧音'),
 ('姜文', '巩俐'),
 ('郑皓原', '刘筱筱'),
 ('周星驰', '林青霞'),
 ('钟汉良', '刘筱筱'),
 ('苗侨伟', '石修'),
 ('巩俐', '滕汝骏'),
 ('潘迎紫', '顾冠忠'),
 ('刘德华', '林子祥'),
 ('梁朝伟', '章子怡'),
 ('金城武', '梁朝伟'),
 ('佘诗曼', '薛家燕'),
 ('陈奕迅', '卢巧音'),
 ('陈道明', '秦岚'),
 ('张国荣', '莫文蔚'),
 ('萧芳芳', '乔宏'),
 ('周星驰', '朱茵'),
 ('尹扬明', '陈秀雯'),
 ('张国荣', '邱淑贞'),
 ('万梓良', '吕良伟'),
 ('张智霖', '郭可盈'),
 ('黄日华', '江华'),
 ('巩俐', '姜武'),
 ('朱茵', '魏骏杰'),
 ('洪欣', '蔡少芬'),
 ('郑丹瑞', '周秀兰'),
 ('袁咏仪', '刘嘉玲'),
 ('葛优', '浅野忠信'),
 ('刘筱筱', '屈菁菁'),
 ('黎明', '温兆伦'),
 ('张国荣', '巩俐'),
 ('孙俪', '陈建斌'),
 ('温兆伦', '刘嘉玲'),
 ('袁咏仪', '张国荣'),
 ('张国荣', '张柏芝'),
 ('周星驰', '吴孟达'),
 ('周润发', '钟楚红'),
 ('乔宏', '罗家英'),
 ('萧芳芳', '李嘉欣'),
 ('严屹宽', '张翰'),
 ('张国立', '张涵予'),
 ('金城武', '梁咏琪'),
 ('陈建斌', '蔡少芬'),
 ('李嘉欣', '金城武'),
 ('张国荣', '钟楚红'),
 ('

In [24]:
import networkx as nx

# 创建 Graph
G = nx.Graph()
# 所有节点
nodes = []
# 所有边
edges = []
# 统计所有名字的组合
for comb in itertools.combinations(all_names, 2):
    # 两个演员合作过，才添加边
    if (comb[0], comb[1]) in has_cooperated_actors or (comb[1], comb[0]) in has_cooperated_actors:
        node1 = comb[0]
        node2 = comb[1]
        nodes.append(node1)
        nodes.append(node2)
        edges.append((node1, node2))
# 去除重复节点
nodes = list(set(nodes))
# 添加节点和边
G.add_nodes_from(nodes)
G.add_edges_from(edges)

# 节点对应到图上的坐标 (x, y)
pos = nx.spring_layout(G)

In [36]:
pos

{'章子怡': array([ 0.1036821 , -0.09094797]),
 '张翰': array([ 0.62447267, -0.66774777]),
 '江华': array([0.12572316, 0.33757467]),
 '张柏芝': array([ 0.08458105, -0.05878481]),
 '邱淑贞': array([ 0.02720898, -0.03744797]),
 '陶大宇': array([0.04783227, 0.20615377]),
 '黄杏秀': array([0.17060385, 0.4873489 ]),
 '张涵予': array([ 0.16667687, -0.30885424]),
 '朱茵': array([-0.14947316,  0.07683412]),
 '张延': array([0.01857371, 0.29291456]),
 '吴岱融': array([0.15971442, 0.09111466]),
 '郑丹瑞': array([0.1087752 , 0.03079496]),
 '刘嘉玲': array([0.02582046, 0.06793434]),
 '林峯': array([0.19390422, 0.24186535]),
 '莫文蔚': array([ 0.00405932, -0.00105833]),
 '陈浩民': array([0.15390777, 0.31019878]),
 '林倩萍': array([-0.24032241,  0.13641366]),
 '温兆伦': array([-0.03584602,  0.16303195]),
 '汤镇业': array([0.13857486, 0.38909676]),
 '袁咏仪': array([-0.05820099,  0.01041767]),
 '伍咏薇': array([0.10271908, 0.31999121]),
 '陈秀雯': array([0.11365911, 0.07963025]),
 '郑皓原': array([-0.37933873, -0.27255101]),
 '赵薇': array([-0.11239867, -0.06309777])

In [30]:
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go
# 绘制边的轨迹
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# 遍历每一条边，根据上文获得的每一个节点的坐标，连接两个节点
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

# 绘制点
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

# 遍历每一个点
for node in G.nodes():
    x, y = pos[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

In [33]:
# 遍历邻接列表，对应了 节点->一条边的另一个节点和权重
for node, adjacencies in enumerate(G.adjacency()):
    name = adjacencies[0]
    # 计算一个演员演过电影的平均得分
    works = [x for x in actors_data.values() if x['name'] == name][0]['works']
    avg_rate = sum([x['subject']['rating']['average']
                    for x in works])/len(works)
    # 每个节点的颜色
    node_trace['marker']['color'] += tuple([avg_rate])
    node_info = '{} {}'.format(name, avg_rate)
    # 每一个节点添加标注信息（名字, 平均评分）
    node_trace['text'] += tuple([node_info])

In [23]:
# 绘制图像
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                title='<br>周星驰为中心的 100 个演员关系',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20, l=5, r=5, t=40),
#                 annotations=[dict(
#                     text="周星驰为中心的 100 个演员关系",
#                     showarrow=False,
#                     xref="paper", yref="paper",
#                     x=0.005, y=-0.002)],
                xaxis=dict(showgrid=False, zeroline=False,
                           showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

iplot(fig, filename='周星驰为中心的 100 个演员关系')